# Clustering Crypto

In [118]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [163]:
# Load the crypto_data.csv dataset.
file_path = "Resources/crypto_data.csv"
cryptodata_df = pd.read_csv(file_path, index_col = 0)
cryptodata_df.head(10)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [120]:
# Keep all the cryptocurrencies that are being traded.
tradedcrypto_df = cryptodata_df[cryptodata_df.IsTrading.eq(True)]
tradedcrypto_df.shape


(1144, 6)

In [121]:
tradedcrypto_df = cryptodata_df.loc[cryptodata_df.IsTrading].reset_index(drop=True)
tradedcrypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
1139,Super Zero,Ethash,True,PoW,NaN,1000000000
1140,UOS,SHA-256,True,DPoI,NaN,1000000000
1141,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1142,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [122]:
# Keep all the cryptocurrencies that have a working algorithm.
tradedcrypto_df.Algorithm.value_counts()
tradedcrypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015 coin,X11,True,PoW/PoS,NaN,0
7,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [123]:
tradedcrypto_df.Algorithm.value_counts()

Scrypt                   394
X11                      182
SHA-256                  121
X13                       54
PoS                       42
                        ... 
VeChainThor Authority      1
Ouroboros                  1
POS 2.0                    1
Proof-of-BibleHash         1
TRC10                      1
Name: Algorithm, Length: 89, dtype: int64

In [124]:
# Remove the "IsTrading" column. 
tradedcrypto_df.drop(['IsTrading'], axis = 1, inplace = True)
tradedcrypto_df.head(10)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
6,2015 coin,X11,PoW/PoS,NaN,0
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
9,Litecoin,Scrypt,PoW,6.303924e+07,84000000


In [125]:
# Remove rows that have at least 1 null value.
filteredcrypto_df = tradedcrypto_df.dropna(how = "any").reset_index(drop = True)
filteredcrypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
2,808,SHA-256,PoW/PoS,0.000000e+00,0
3,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
4,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
5,Ethereum,Ethash,PoW,1.076842e+08,0
6,Litecoin,Scrypt,PoW,6.303924e+07,84000000
7,Dash,X11,PoW/PoS,9.031294e+06,22000000
8,Monero,CryptoNight-V7,PoW,1.720114e+07,0
9,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000


In [126]:
# Keep the rows where coins are mined.
minedcoins_df = filteredcrypto_df.loc[filteredcrypto_df.TotalCoinsMined > 0].reset_index(drop=True)
minedcoins_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
2,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
3,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
4,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
527,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
528,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
529,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
530,Horizen,Equihash,PoW,7.296538e+06,21000000


In [127]:
# Create a new DataFrame that holds only the cryptocurrencies names.
cryptonames_df = minedcoins_df.loc[:, ['CoinName']]
cryptonames_df.head(10)

,CoinName
0,42 Coin
1,404Coin
2,EliteCoin
3,Bitcoin
4,Ethereum
5,Litecoin
6,Dash
7,Monero
8,Ethereum Classic
9,ZCash


In [128]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
cluster_crypto = minedcoins_df.drop(['CoinName'], axis = 1)
cluster_crypto.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
1,Scrypt,PoW/PoS,1.055185e+09,532000000
2,X13,PoW/PoS,2.927942e+10,314159265359
3,SHA-256,PoW,1.792718e+07,21000000
4,Ethash,PoW,1.076842e+08,0
5,Scrypt,PoW,6.303924e+07,84000000
6,X11,PoW/PoS,9.031294e+06,22000000
7,CryptoNight-V7,PoW,1.720114e+07,0
8,Ethash,PoW,1.133597e+08,210000000
9,Equihash,PoW,7.383056e+06,21000000


In [129]:
# Use get_dummies() to create variables for text features.
crypto_features = pd.get_dummies (cluster_crypto, columns = ['Algorithm', 'ProofType'])
crypto_features.head(10)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
0,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,7.383056e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [130]:
# Standardize the data with StandardScaler().
standardize_df = StandardScaler().fit_transform(crypto_features)
print(standardize_df[:1])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [131]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components = 3)
pca_reduce = pca.fit_transform(standardize_df)
pca_reduce

array([[-0.33128476,  0.97996485, -0.43922139],
       [-0.3146092 ,  0.98006089, -0.43952816],
       [ 2.31205821,  1.55323538, -0.47642803],
       ...,
       [ 0.31889036, -2.27998842,  0.27881009],
       [-0.16031981, -1.91770849,  0.35668061],
       [-0.29237127,  0.79924903, -0.21640151]])

In [132]:
# Create a DataFrame with the three principal components.
pca_df = pd.DataFrame(data = pca_reduce, columns = ['PC1','PC2','PC3'])
pca_df.head(10)

,PC1,PC2,PC3
0,-0.331285,0.979965,-0.439221
1,-0.314609,0.980061,-0.439528
2,2.312058,1.553235,-0.476428
3,-0.148839,-1.285179,0.157050
4,-0.155818,-2.017783,0.346553
5,-0.170270,-1.114449,-0.016199
6,-0.391345,1.226283,-0.380271
7,-0.154440,-2.187645,0.334879
8,-0.154259,-2.017891,0.346538
9,-0.160319,-1.917708,0.356681


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [133]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1,11))

for i in k:
        km = KMeans(n_clusters = i, random_state = 0)
        km.fit(pca_df)
        inertia.append(km.inertia_)


D:\Code\envs\PythonData\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  "KMeans is known to have a memory leak on Windows "


Running K-Means with `k=4`

In [134]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pca_df)

# Predict clusters
predict = model.predict(pca_df)

In [109]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.merge(minedcoins_df, pca_df, left_index=True, right_index=True)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = cryptonames_df.CoinName

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = predict

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,Class
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42,-0.335657,0.964231,-0.545413,0
1,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.319035,0.963785,-0.545628,0
2,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359,2.279136,1.453817,-0.682279,0
3,Bitcoin,SHA-256,PoW,1.792718e+07,21000000,-0.157216,-1.209513,0.163623,3
4,Ethereum,Ethash,PoW,1.076842e+08,0,-0.150882,-1.913855,0.366779,3
5,Litecoin,Scrypt,PoW,6.303924e+07,84000000,-0.158830,-1.090176,0.025608,3
6,Dash,X11,PoW/PoS,9.031294e+06,22000000,-0.398297,1.143937,-0.494246,0
7,Monero,CryptoNight-V7,PoW,1.720114e+07,0,-0.149462,-2.082848,0.454366,3
8,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000,-0.149328,-1.914015,0.366772,3
9,ZCash,Equihash,PoW,7.383056e+06,21000000,-0.138888,-1.970238,0.320469,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [161]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df,
    x="PC1",
    y="PC2",
    z="PC3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name="CoinName",
    hover_data=["Algorithm"],
    title="3D Crypto Clusters with PCA",
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [148]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=["Algorithm", "ProofType", "TotalCoinSupply", "Class"])

:Table   [Algorithm,ProofType,TotalCoinSupply,Class]

In [155]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {len(clustered_df)}tradeable crypto coins available.")

There are 532tradeable crypto coins available.


In [156]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
trade_scaled = MinMaxScaler().fit_transform(
            clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])
print(trade_scaled[0:5])

[[4.20000000e-11 0.00000000e+00]
 [5.32000000e-04 1.06585544e-03]
 [3.14159265e-01 2.95755135e-02]
 [2.10000000e-05 1.81084216e-05]
 [0.00000000e+00 1.08773140e-04]]


In [158]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
scaledclusters_df=pd.DataFrame(trade_scaled, columns=['TotalCoinSupply', 'TotalCoinsMined'])

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
scaledclusters_df['CoinName'] = clustered_df.CoinName

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
scaledclusters_df['Class'] = clustered_df.Class

scaledclusters_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
0,4.200000e-11,0.000000,42 Coin,0
1,5.320000e-04,0.001066,404Coin,0
2,3.141593e-01,0.029576,EliteCoin,0
3,2.100000e-05,0.000018,Bitcoin,3
4,0.000000e+00,0.000109,Ethereum,3
5,8.400000e-05,0.000064,Litecoin,3
6,2.200000e-05,0.000009,Dash,0
7,0.000000e+00,0.000017,Monero,3
8,2.100000e-04,0.000115,Ethereum Classic,3
9,2.100000e-05,0.000007,ZCash,3


In [160]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
scaledclusters_df.hvplot.scatter(
                                x='TotalCoinsMined',
                                y='TotalCoinSupply',
                                hover_cols='CoinName')


:Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)